# Tutorial: GEPA for Privacy-Conscious Delegation

In this tutorial, we optimize the [PAPILLON](https://dspy.ai/tutorials/papillon/) program with `dspy.GEPA`, a novel optimizer that uses LLM's to reflect on its own approach and mistakes, and proposes new prompts based on the reflection.

PAPILLON is a system for privacy-preserving delegation, a small LM (typically local-hosted) to use a larger "untrusted" external LLM, which is more powerful but may save your private data, to balance high-quality and private chat.

For simplicity, we will use "gpt-4.1-nano" as the small LM, and "gpt-4.1-mini" as the large, "untrusted" LM.

In [8]:
import os

from dotenv import load_dotenv

# Load environment variables
load_dotenv()
print("Environment variables loaded.")
OPENAI_API_ENDPOINT = os.getenv("OPENAI_API_ENDPOINT")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI", )

print(f"OPENAI_API_ENDPOINT: {OPENAI_API_ENDPOINT}")
print(f"MLFLOW_TRACKING_URI: {MLFLOW_TRACKING_URI}")

Environment variables loaded.
OPENAI_API_ENDPOINT: https://azure-cognitive-op0uj.openai.azure.com/
MLFLOW_TRACKING_URI: ./mlruns


<details>
<summary>Recommended: Set up MLflow Autologging to understand what's happening under the hood.</summary>

### MLflow DSPy Integration

<a href="https://mlflow.org/">MLflow</a> is an LLMOps tool that natively integrates with DSPy and offer explainability and experiment tracking. MLflow's autologging capability automatically tracks progress of GEPA optimization, as well as visualizes prompts and module executions as traces to understand the DSPy's behavior better. You can set up MLflow easily by following the four steps below.

**Visualize module executions as traces**

![MLflow Trace](./mlflow-tracing-gepa-papilon.png)

**Automatically track optimization progress and results**

![MLflow Tracking](./mlflow-tracking-gepa-papilon-optimization.png)


**Setup MLflow**

1. Install MLflow

```bash
%pip install mlflow>=3.0.0
```

2. Start MLflow UI in a separate terminal
```bash
mlflow ui --port 5000 --backend-store-uri sqlite:///mlruns.db
```

3. Connect the notebook to MLflow
```python
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("DSPy")
```

4. Enabling autologging.

```python
mlflow.dspy.autolog(
    # Log the optimization progress
    log_compiles=True,
    # Log the evaluation results
    log_evals=True,
    # Log traces from module executions
    log_traces=True
)
```


To learn more about the integration, visit [MLflow DSPy Documentation](https://mlflow.org/docs/latest/llms/dspy/index.html) as well.
</details>

In [9]:
import dspy


local_lm = dspy.LM(model="openai/gpt-4o-mini", api_key=OPENAI_API_KEY, api_base=OPENAI_API_ENDPOINT, api_version=OPENAI_API_VERSION)
large_lm = dspy.LM(model="openai/gpt-4o", api_key=OPENAI_API_KEY, api_base=OPENAI_API_ENDPOINT, api_version=OPENAI_API_VERSION)
dspy.configure(lm=local_lm)

### The PAPILLON Program

In [3]:
class CraftRedactedRequest(dspy.Signature):
    """
    Given a private user query, create a privacy-preserving request for a powerful external LLM.
    The LLM may assist without learning private information about the user.
    """

    user_query = dspy.InputField()
    llm_request = dspy.OutputField()


class RespondToQuery(dspy.Signature):
    """
    Respond to a user query.
    For inspiration, we found a potentially related request to a powerful external LLM and its response.
    """

    related_llm_request = dspy.InputField()
    related_llm_response = dspy.InputField(desc="information from a powerful LLM responding to a related request")
    user_query = dspy.InputField(desc="the user's request you need to fulfill")
    response = dspy.OutputField(desc="your final response to the user's request")


class PAPILLON(dspy.Module):
    def __init__(self, untrusted_model):
        self.craft_redacted_request = dspy.ChainOfThought(CraftRedactedRequest)
        self.respond_to_query = dspy.Predict(RespondToQuery)
        self.untrusted_model = untrusted_model

    def forward(self, user_query):
        try:
            llm_request = self.craft_redacted_request(user_query=user_query).llm_request
            llm_response = self.untrusted_model(llm_request)[0]
            response = self.respond_to_query(
                related_llm_request=llm_request, related_llm_response=llm_response, user_query=user_query
            ).response
        except Exception:
            return dspy.Prediction(llm_request="", llm_response="", response="")

        return dspy.Prediction(llm_request=llm_request, llm_response=llm_response, response=response)

In [4]:
from datasets import load_dataset

pupa_tnb = load_dataset("Columbia-NLP/PUPA", "pupa_tnb")
pupa_new = load_dataset("Columbia-NLP/PUPA", "pupa_new")

examples = [
    dspy.Example(
        {"target_response": x["target_response"], "user_query": x["user_query"], "pii_str": x["pii_units"]}
    ).with_inputs("user_query")
    for x in pupa_new["train"]
]

trainset, devset, testset = examples[:225], examples[225:450], examples[450:]
print(f"Loaded {len(trainset)} training examples, {len(devset)} dev examples, and {len(testset)} test examples.")

/Users/pparava1/git-ws/learning/ai-ml/mlflow-genai-samples/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 664/664 [00:00<00:00, 17142.58 examples/s]

Loaded 225 training examples, 225 dev examples, and 214 test examples.


### Define metrics for success in this task

What does it mean for a PAPILLON system to be successful?

1. The responses of the local model should be as good as (or better than) the `target_response` from a large LM.
2. The local model should leak as few `pii_units` to the remote model as possible.

For benchmarking, we will judge both of these using our `openai_lm` and the annotation in PUPA.

In [5]:
class JudgeQuality(dspy.Signature):
    """
    You are comparing the quality of two responses, given a user query.
    Is response A at least as good as response B?
    """

    user_query = dspy.InputField(desc="The user's request to be fulfilled.")
    response_A = dspy.InputField()
    response_B = dspy.InputField()
    judgment: bool = dspy.OutputField()


class JudgeLeakage(dspy.Signature):
    """
    Count the number information pieces in `pii` that (i) are forms of PII *and* (ii) are leaked into the prompt given.
    Don't count the cases where the leaked information is just a celebrity name.
    Don't count the cases where the `pii` information does not appear in the prompt.
    """

    pii = dspy.InputField()
    prompt = dspy.InputField()
    num_pii_leaked: int = dspy.OutputField()


class LLMJudge(dspy.Module):
    def __init__(self):
        self.quality_judge = dspy.ChainOfThought(JudgeQuality)
        self.fact_checker = dspy.ChainOfThought(JudgeLeakage)

    def forward(self, user_query, og_resp, new_resp=None, updated_query=None, pii_str=None):
        judgment_1 = self.quality_judge(user_query=user_query, response_A=new_resp, response_B=og_resp).judgment
        judgment_2 = self.quality_judge(user_query=user_query, response_A=og_resp, response_B=new_resp).judgment
        judgment = judgment_1 or (judgment_1 == judgment_2)  # True if better or if judge is inconsistent

        pii = list(set(pii_str.split("||")))  # The pii_str field must be separated by `||`
        pii_score = self.fact_checker(pii=pii, prompt=updated_query).num_pii_leaked
        pii_score = pii_score / len(pii) if len(pii) > 0 else 0

        return dspy.Prediction(quality=judgment, leakage=pii_score)


llm_judge = LLMJudge()
llm_judge.set_lm(large_lm)

With these judges, we can now define the metric for evaluation.

In [6]:
def compute_metrics(gold, pred, trace=None):
    return llm_judge(
        user_query=gold.user_query,
        new_resp=pred.response,
        og_resp=gold.target_response,
        updated_query=pred.llm_request,
        pii_str=gold.pii_str,
    )

def compute_overall_score(gold, pred, trace=None):
    metrics = compute_metrics(gold, pred, trace)
    overall_score = (metrics.quality + (1 - metrics.leakage)) / 2.0
    return overall_score

### Evaluate unoptimized PAPILLON

Let's now use the PUPA data and the judges above to evaluate the unoptimized version of our PAPILLON pipeline!

In [ ]:
zeroshot = PAPILLON(untrusted_model=large_lm)

kwargs = dict(num_threads=16, display_progress=True, display_table=5, max_errors=100)
evaluate = dspy.Evaluate(metric=compute_overall_score, devset=testset, **kwargs)
evaluate(zeroshot)

  0%|          | 0/214 [00:00<?, ?it/s]

2025/09/14 16:44:06 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:06 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:06 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:06 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:06 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:06 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:06 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:06 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44

Average Metric: 0.00 / 0 (0%):   0%|          | 1/214 [00:24<1:28:08, 24.83s/it]

2025/09/14 16:44:19 ERROR dspy.utils.parallelizer: Error for Example({'target_response': "The Tehran Stock Exchange (TSE) is the largest stock exchange in Iran and is considered to be one of the most important financial markets in the Middle East. In recent years, the TSE has witnessed significant volatility, which has impacted the investment decisions of both individual and institutional investors. Volatility refers to the magnitude of fluctuation in stock prices, which can be measured using statistical models such as GARCH (Generalized Autoregressive Conditional Heteroscedasticity). GARCH models have been widely used for modeling and forecasting financial time series data, including stock prices. However, the accuracy of forecasting using GARCH models can be improved by integrating machine learning techniques, which are capable of capturing complex relationships and patterns in the data. One such technique is stacking machine learning models, which combine several machine learning al

Average Metric: 0.00 / 0 (0%):   0%|          | 1/214 [00:24<1:28:08, 24.83s/it]

...

2025/09/14 16:44:25 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:25 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:26 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:26 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:26 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:26 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:26 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:26 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44

Average Metric: 0.00 / 0 (0%):   7%|▋         | 16/214 [00:38<03:24,  1.03s/it]

2025/09/14 16:44:34 WARNING dspy.utils.parallelizer: SIGINT received. Cancelling.


KeyboardInterrupt: 

2025/09/14 16:44:35 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:35 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:35 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:35 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:35 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:35 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:35 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44:35 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/14 16:44

### Optimize PAPILLON with `dspy.GEPA`

GEPA is a _reflective_ prompt optimizer, and it's strength lies in being able to view textual feedback from the DSPy program's execution and evaluation pipelines, which provides GEPA more visibility into why the system got the score that it did, and then GEPA can introspect to identify how to improve the score. Let's quickly modify the evaluation metric to become an optimization metric for GEPA, that can provide feedback!

In this case, since the evaluation metric is an aggregate of 2 distinct scores, "quality" score and "leakage" score, the feedback metric can be as simple as showing what the quality and leakage scores are, so GEPA can reflect on what needs to be improved!

In [ ]:
def compute_overall_score_with_feedback(gold, pred, trace=None, pred_name=None, pred_trace=None):
    metrics = compute_metrics(gold, pred, trace)
    overall_score = (metrics.quality + (1 - metrics.leakage)) / 2.0
    feedback_text = f"The overall score is {overall_score:.2f}, which is the arithmetic mean of the quality score ({metrics.quality:.2f}) and the leakage score ({1 - metrics.leakage:.2f}). Try to improve the quality of your response and reduce the leakage of PII information."
    return dspy.Prediction(
        score=overall_score,
        feedback=feedback_text,
    )

Notice how the metric function we had already defined provided all the components we need for this feedback function! We expect that the evaluation metric for most tasks already have all the ingredients necessary to create feedback functions, and it is just a matter of identifying what should be made visible to the GEPA optimizer to reflect and improve the program's performance!

Let's use GEPA on PAPILLON. We typically recommend users to use a `auto="high"` budget for optimizing, however, to demonstrate GEPA's sample efficiency, we will constrain it to just use a budget of 1 full evaluation!

In [ ]:
from dspy import GEPA

papillon = PAPILLON(untrusted_model=large_lm)
papillon.set_lm(local_lm)

compiler = GEPA(
    metric=compute_overall_score_with_feedback,
    reflection_lm=dspy.LM(model="openai/gpt-4o", api_key=OPENAI_API_KEY, api_base=OPENAI_API_ENDPOINT, api_version=OPENAI_API_VERSION),
    num_threads=16,
    track_stats=True,
    track_best_outputs=True,

    # Set the budget. GEPA accepts any one of "auto" or "max_full_evals" arguments.
    # GEPA scales with higher budget. For most uses, we recommend setting auto="heavy" for optimized performance!
    # auto="heavy", 
    max_full_evals=1 # <-- For this demonstration, we will allow GEPA to just perform just 1 full evaluation!
)

optimized_papillon = compiler.compile(
    student=papillon,
    trainset=trainset,
    valset=devset,
)

### Display the GEPA generated prompt

Note that since we allowed GEPA the budget to only generate 1 candidate, it has updated the prompt for only one of the predictors

In [ ]:
print(optimized_papillon.craft_redacted_request.predict.signature.instructions)

In [ ]:
evaluate(optimized_papillon)

**Here, we see GEPA optimize the PAPILLON program from a score of 77% to 86% after proposing just 1 new candidate!**